# 4. Численное решение интегрального уравнения Фредгольма 2-го рода

In [111]:
import math
import numpy as np
import sympy as sp
from pandas import DataFrame as df

$ u(x) - 0.6 \int_0^1{ch(xy)u(y)dy} = x - 0.6 $

In [112]:
def f(x):
    return x - 0.6

def H(x, y):
    return -0.6 * math.cosh(x * y)

## Метод замены ядра на вырожденное

Разложенем -0.6ch(xy) в ряд Маклорена двух переменных
$$ \widetilde{H^n}(x, y) = -0.6 \cdot \sum_{k = 0}^{n-1}{x^{2k} \cdot \frac{y^{2k}}{2k!}} $$
$$ \alpha_k(x) = -0.6 \cdot x^{2k},\, \beta_k(y) = \frac{y^{2k}}{2k!} $$

In [113]:
def u(n):
    A = [[0] * n for i in range(n)]
    b = [0] * n
    for i in range(0, n):
        y = sp.Symbol('y')
        integral = sp.integrate(y ** (2 * i) / sp.factorial(2 * i) * (y - 0.6), y)
        by = sp.lambdify(y, integral, 'math')
        b[i] = by(1) - by(0)
        for j in range(0, n):
            integral = sp.integrate(-0.6 * y ** (2*(i+j)) / sp.factorial(2 * i), y)
            gammaij = sp.lambdify(y, integral, 'math')
            A[i][j] = (1 if i == j else 0) - (gammaij(1) - gammaij(0))
    c = np.linalg.solve(A, b)
    return lambda x: f(x) + sum([c[i] * -0.6 * x ** (2 * i) for i in range(n)])

In [114]:
u3 = u(3)
u4 = u(4)

In [115]:
table = df(data={
    'x': ['$\widetilde{u^3}$', '$\widetilde{u^4}$'],
    '0': [u3(0), u4(0)],
    '0.5': [u3(0.5), u4(0.5)],
    '1': [u3(1), u4(1)]
})
table.style.hide_index()

x,0,0.5,1
$\widetilde{u^3}$,-0.560171,-0.064716,0.420675
$\widetilde{u^4}$,-0.560169,-0.064714,0.420642


$\widetilde{\Delta}$:

In [116]:
delta = max([abs(u4(x) - u3(x)) for x in [i / 100000 for i in range(100000)]])
delta

3.29136295962873e-05

## Метод механических квадратур

In [117]:
def u(n):
    h = 1/n
    
    def x(i):
        return h * i + h / 2
    
    g = [f(x(i)) for i in range(n)]
    D = [[0] * n for i in range(n)]
    for i in range(n):
        for j in range(n):
            D[i][j] = (1 if i == j else 0) - h * H(x(i), x(j))
    
    z = np.linalg.solve(D, g)
    return lambda y: f(y) + sum([h * H(y, x(i)) * z[i] for i in range(n)])

In [119]:
m = 1
ms = [1]
while max([abs(u(2 * m)(x) - u(m)(x)) for x in [0, 0.5, 1]]) >= delta:
    m *= 2
    ms.append(m)
table = df(data={
    'm\\x': [i for i in ms] + ['решение пред.пункта'],
    '0': [u(i)(0) for i in ms] + [u4(0)],
    '0.5': [u(i)(0.5) for i in ms] + [u4(0.5)],
    '1': [u(i)(1) for i in ms] + [u4(1)]
})
table.style.hide_index()

m\x,0,0.5,1
1,-0.562937,-0.061772,0.441794
2,-0.561002,-0.064150,0.425865
4,-0.560386,-0.064585,0.421940
8,-0.560224,-0.064683,0.420966
16,-0.560183,-0.064706,0.420723
32,-0.560172,-0.064712,0.420662
решение пред.пункта,-0.560169,-0.064714,0.420642


In [120]:
table = df(data={
    'x': [0, 0.5, 1],
    '$u^{2m}(x) - u^{m}(x)$': [u(2 * m)(x) - u(m)(x) for x in [0, 0.5, 1]]
})
table.style.hide_index()
table.style.set_properties(subset=['$u^{2m}(x) - u^{m}(x)$'], width='150px')

,x,$u^{2m}(x) - u^{m}(x)$
0,0.000000,0.000003
1,0.500000,-0.000001
2,1.000000,-0.000015
